In [1]:
setwd("/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.1. Data Imputation/Input")
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.1. Data Imputation/Output')
cur_date = "020922"


library(imputeLCMD)
library(tidyverse)
library(janitor)

#reading in files
mf_df = read_csv("080321_NoGapFillNoScaling_Dmp3872.csv")
cytokine_df = read_csv("CytokineData_61Subjects_080421.csv")

Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix

Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following object is masked from ‘package:Matrix’:

    which


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
   

Both datasets having missing data that will be imputed using the QRILC method.

# Molecular Feature Dataset Imputation

In [2]:
head(mf_df)

MF_Number,NS_070,NS_110,NS_143,NS_152,NS_187,NS_196,NS_215,NS_216,NS_217,⋯,NA_SB5,NA_SB6,NA_SB7,NA_SC1,NA_SC2,NA_SC3,NA_SC4,NA_SC5,NA_SC6,NA_SC7
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
MF_2,1116206475,2608024693,10436125,3733023738,832538895,7117302385,3479543793,886059325,4829905676,⋯,NA,NA,NA,NA,750551.7,487606.7,942984,NA,NA,NA
MF_3,NA,1688508261,5981200,2631091440,NA,5794479646,2855197795,NA,4654806576,⋯,NA,NA,NA,NA,742069.1,NA,389778,NA,418340.2,NA
MF_4,383833198,1374010136,7153567,2629366777,533802873,4533282195,1919783356,353083003,3228249341,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
MF_5,1680099,NA,NA,NA,NA,NA,1573718,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
MF_6,NA,NA,NA,3170821,NA,NA,324521665,NA,7750065,⋯,NA,405819.5,489580.9,NA,289593.1,NA,NA,NA,NA,950554.9
MF_7,129935373,546737402,NA,1191854581,NA,2417054513,1345599792,195011396,2024878820,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


# Cytokine Dataset Imputation

In [3]:
head(cytokine_df)

SampleID,Gender,ExposureGroup,ID,IFNgamma,IL10,IL12p70,IL13,IL1beta,IL2,⋯,Eotaxin,Eotaxin3,IL8_1,IP10,MCP10,MCP4,MDC,MIP1alpha,MIP1beta,TARC
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
CA39,Female,ecig_user,Ecig_39,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,⋯,358.19662,NA,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
CA41,Female,ecig_user,Ecig_41,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,⋯,267.15489,69.256817,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
CA44,Female,ecig_user,Ecig_44,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,⋯,103.50569,8.724533,11221.13,7109.3784,147.73303,NA,167.32860,46.98751,101.25635,35.541708
CA45,Female,ecig_user,Ecig_45,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,⋯,298.85050,8.438962,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
CA47,Female,ecig_user,Ecig_47,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,⋯,67.18448,5.487179,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
CA48,Female,ecig_user,Ecig_48,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,⋯,25.07486,2.151572,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


In [4]:
cytokine_df = cytokine_df %>%
    #removing unecessary metadata cols
    select(!c(SampleID, Gender, ExposureGroup)) %>%
    #removing rows without ids
    filter(is.na(ID)== FALSE) %>%
    column_to_rownames("ID")

head(cytokine_df)

,IFNgamma,IL10,IL12p70,IL13,IL1beta,IL2,IL4,IL6,IL8,TNFalpha,⋯,Eotaxin,Eotaxin3,IL8_1,IP10,MCP10,MCP4,MDC,MIP1alpha,MIP1beta,TARC
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ecig_39,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,0.42455866,13.075670,2317.1204,5.9501019,⋯,358.19662,NA,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
Ecig_41,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,0.71329848,19.032378,2340.2937,11.3628057,⋯,267.15489,69.256817,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
Ecig_44,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,0.49649954,9.266373,2233.1268,6.3306852,⋯,103.50569,8.724533,11221.13,7109.3784,147.73303,NA,167.32860,46.98751,101.25635,35.541708
Ecig_45,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,0.02914528,6.395885,1889.0348,1.9479068,⋯,298.85050,8.438962,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
Ecig_47,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,0.08862797,5.244501,2074.0063,3.5847722,⋯,67.18448,5.487179,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
Ecig_48,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,0.14116411,2.476904,823.3077,0.5468069,⋯,25.07486,2.151572,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


In [5]:
#There are two IL8 cytokines specifying two ranges of detection. We only want to consider the upper range, 
#thus the lower range is dropped
cyt <- as.data.frame(t(cytokine_df))
cyt <- cyt %>% rownames_to_column("cytokine")
cyt[9,1] <- "IL8_drop"
cyt[23,1] <- "IL8"
cytokine_df <- cyt %>% filter(cytokine!="IL8_drop") %>% column_to_rownames("cytokine")
head(cytokine_df)

,Ecig_39,Ecig_41,Ecig_44,Ecig_45,Ecig_47,Ecig_48,Ecig_214,Ecig_222,Ecig_223,Ecig_229,⋯,NS_217,NS_220,NS_224,NS_227,NS_234,NS_242,NS_245,NS_246,NS_263,NS_264
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
IFNgamma,5.369561,11.612812,4.160319,0.8881171,1.4481966,0.8494440,2.2133006,12.194983,10.359429,11.274641,⋯,1.4100626,2.2885988,1.6232601,12.568316,1.9416090,10.337784,7.890241,0.7175676,7.147723,2.0700327
IL10,2.992854,9.060027,2.034095,0.4310717,0.4997130,0.4240552,1.2942145,4.519250,2.141442,14.467577,⋯,0.3368341,0.6621680,0.4240552,22.625324,0.3994781,4.156594,3.000276,0.1950644,2.339561,0.3868263
IL12p70,1.670016,3.371081,1.565569,0.3063518,0.8622316,0.2785117,0.7038064,5.050170,1.880607,4.079984,⋯,0.5284839,0.7517281,0.8180437,3.124857,0.5377213,4.800905,1.950156,0.2952182,1.814701,0.5303315
IL13,48.368132,67.963220,50.771205,25.2390558,36.8147135,14.5127220,34.6004791,83.282335,61.603704,85.073378,⋯,10.9902839,31.6987943,33.3564823,94.411251,13.3227644,89.613874,69.582659,20.0421143,59.211105,23.3918189
IL1beta,24.117761,167.431877,8.870868,40.4776501,15.7558575,9.2116799,53.3450039,98.602727,92.383512,180.302138,⋯,9.4157549,21.4534240,27.8493328,927.559496,11.5052411,89.453532,104.084531,22.8313411,53.797197,17.8222380
IL2,5.265719,12.683771,2.483751,0.5200777,0.7766565,0.2684367,1.8460707,9.060809,4.800959,11.239145,⋯,0.7175273,0.8714269,0.2814659,14.667649,0.7407722,8.725433,5.653496,0.4216734,4.668523,0.3614672


In [6]:
#Convert cytokine to binary (if the value is missing 0, if the value is present 1)
cyt_missing <- cytokine_df %>% mutate_all(~replace(.,!is.na(.), 1)) %>%
  mutate_all(~replace(., is.na(.), 0)) %>%
  mutate_all(function(x) as.numeric(as.character(x)))
 
#Identify cytokines that are detected in at least 25% of samples
features_to_keep <- cyt_missing %>% adorn_totals(where = "col",... = colnames(cyt_missing)) %>% 
  filter(Total >= ncol(cytokine_df)*.25)
head(features_to_keep)        

,Ecig_39,Ecig_41,Ecig_44,Ecig_45,Ecig_47,Ecig_48,Ecig_214,Ecig_222,Ecig_223,Ecig_229,⋯,NS_220,NS_224,NS_227,NS_234,NS_242,NS_245,NS_246,NS_263,NS_264,Total
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
IFNgamma,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,59
IL10,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,59
IL12p70,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,59
IL13,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,59
IL1beta,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,59
IL2,1,1,1,1,1,1,1,1,1,1,⋯,1,1,1,1,1,1,1,1,1,59


In [7]:
#Filter the cytokine data for the cytokines that should be retained based on previous identification step
filtered_cytokine_matrix = cytokine_df %>% 
    rownames_to_column("cytokine") %>%
    filter(cytokine %in% rownames(features_to_keep)) %>%
    column_to_rownames("cytokine") %>%
    t()
head(filtered_cytokine_matrix)

,IFNgamma,IL10,IL12p70,IL13,IL1beta,IL2,IL4,IL6,TNFalpha,GMCSF,⋯,Eotaxin,Eotaxin3,IL8,IP10,MCP10,MCP4,MDC,MIP1alpha,MIP1beta,TARC
Ecig_39,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,0.42455866,13.075670,5.9501019,0.1302776,⋯,358.19662,NA,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
Ecig_41,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,0.71329848,19.032378,11.3628057,1.5269950,⋯,267.15489,69.256817,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
Ecig_44,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,0.49649954,9.266373,6.3306852,0.7731633,⋯,103.50569,8.724533,11221.13,7109.3784,147.73303,NA,167.32860,46.98751,101.25635,35.541708
Ecig_45,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,0.02914528,6.395885,1.9479068,0.2527908,⋯,298.85050,8.438962,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
Ecig_47,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,0.08862797,5.244501,3.5847722,0.2845560,⋯,67.18448,5.487179,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
Ecig_48,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,0.14116411,2.476904,0.5468069,0.0571680,⋯,25.07486,2.151572,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


In [8]:
imputed_QRILC_object = impute.QRILC(filtered_cytokine_matrix)
imputed_cytokine_df = data.frame(imputed_QRILC_object[1])
head(imputed_cytokine_df)

,IFNgamma,IL10,IL12p70,IL13,IL1beta,IL2,IL4,IL6,TNFalpha,GMCSF,⋯,Eotaxin,Eotaxin3,IL8,IP10,MCP10,MCP4,MDC,MIP1alpha,MIP1beta,TARC
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Ecig_39,5.3695611,2.9928536,1.6700162,48.36813,24.117761,5.2657189,0.42455866,13.075670,5.9501019,0.1302776,⋯,358.19662,-80.836525,18547.74,8736.7132,112.48249,10.072081,31.69299,22.02667,46.80070,20.541303
Ecig_41,11.6128116,9.0600267,3.3710810,67.96322,167.431877,12.6837712,0.71329848,19.032378,11.3628057,1.5269950,⋯,267.15489,69.256817,109246.09,7708.0228,255.29369,13.009597,221.57141,52.67214,149.02215,24.715188
Ecig_44,4.1603189,2.0340947,1.5655687,50.77121,8.870868,2.4837514,0.49649954,9.266373,6.3306852,0.7731633,⋯,103.50569,8.724533,11221.13,7109.3784,147.73303,-12.185160,167.32860,46.98751,101.25635,35.541708
Ecig_45,0.8881171,0.4310717,0.3063518,25.23906,40.477650,0.5200777,0.02914528,6.395885,1.9479068,0.2527908,⋯,298.85050,8.438962,14633.80,3045.9940,95.82647,21.200301,196.12405,32.89026,24.92311,19.708475
Ecig_47,1.4481966,0.4997130,0.8622316,36.81471,15.755858,0.7766565,0.08862797,5.244501,3.5847722,0.2845560,⋯,67.18448,5.487179,12078.16,997.9376,66.13370,7.189953,139.46181,16.70862,21.35656,9.885379
Ecig_48,0.8494440,0.4240552,0.2785117,14.51272,9.211680,0.2684367,0.14116411,2.476904,0.5468069,0.0571680,⋯,25.07486,2.151572,4880.69,246.9985,57.50348,3.359783,69.35689,9.56433,15.67118,9.302557


In [ ]:
#exporting data
write.csv(imputed_cytokine_df, paste0(Output,"/", cur_date, "_imputed_cytokine_data.csv"), row.names = TRUE)